<a href="https://colab.research.google.com/github/yuncaich/TFG_Proyect/blob/main/Notebook/Preprocesado_de_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
from os import mkdir

from tqdm import tqdm
import numpy as np
import nibabel as nib
import torch
import torch.nn as nn
from skimage import io
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
import torch.utils.data as data
from PIL import Image

<h1> Variables

In [ ]:
url = "/content/drive/MyDrive/Segmentacion/Imagen_Seccionado_128x128/volume-2"
url_label = "/content/drive/MyDrive/Segmentacion/Label_Seccionado_128x128/volume-2"

path_name = os.listdir(url)
path_name_label = os.listdir(url_label)



<h1> Seccionar imagenes 3D a 2D

In [ ]:
class ImagesCustomDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir_images, root_dir_label, transform=None):
        self.transform = transform
        volumes = list(filter(lambda x: True if 'volume-' in x else False, os.listdir(root_dir_images)))
        self.data = []
        self.targets = []
        for volume_id in volumes:
          img_dir = os.path.join(root_dir_images, volume_id)
          target_dir = os.path.join(root_dir_label, volume_id)
          samples = list(map(lambda x: os.path.join(img_dir, x), sorted(os.listdir(img_dir))))
          labels = list(map(lambda x: os.path.join(target_dir, x), sorted(os.listdir(target_dir))))

          self.data.extend(samples)
          self.targets.extend(labels)

    def __len__(self):
      return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data[idx])
        target = np.load(self.targets[idx])
        if self.transform:
          img=self.transform(img)

        return (img, target)

In [ ]:



def read_nii(filepath):
    '''
    Reads .nii file and returns pixel array
    '''
    ct_scan = nib.load(filepath)
    array   = ct_scan.get_fdata()
    array   = np.rot90(np.array(array))
    return(array)


def segment_image(input_filepath,output_filepath):
  contador = 0
  path_name = os.listdir(input_filepath)

  for file_name in path_name:
    file_name_splited, _ = file_name.split('.')
    output_file_name = output_filepath + "/" + file_name_splited
    if not os.path.exists(output_file_name):
      data = read_nii(input_filepath + "/" + file_name)

      mkdir(output_file_name)
      samples = np.transpose(data, (2, 0, 1))
      
      for i in range(samples.shape[0]):

        res = cv2.resize(data[:,:,i], dsize=(128, 128))
        cv2.imwrite(output_file_name + "/" + str(contador) + ".jpg" , res )
        contador = contador + 1
      contador=0

def interpolate(mask: np.ndarray, size = (128,128)) -> np.ndarray:
  resized = np.array(Image.fromarray(mask*255).resize(size, Image.BILINEAR))
  resized[resized<128] = 0
  resized[resized>=128] = 1 
  return resized

def segment_label(input_filepath,output_filepath):
  contador = 0
  contador_path = 0
  path_name = os.listdir(input_filepath)

  for file_name in path_name:

    cont= file_name.split(".")[0].split("-")[1]
    output_file_name = output_filepath + "/" + "volume-" + cont
    
    if not os.path.exists(output_file_name):
      mask = read_nii(input_filepath + "/" + file_name)
      mask = np.transpose(mask, (2,0,1))
      mkdir(output_file_name)
     
      n_classes = len(np.unique(mask))
      for slice in mask[:]:
        w, h = slice.shape
        np_mask = np.zeros((n_classes-1, w, h))
        resized_np_mask = np.zeros((n_classes-1, 128, 128))
  
        for label in range(1, n_classes):
          idx = np.where(slice==label)

          #print(label, len(idx[0]))
          label_mask = np_mask[label-1] 
          label_mask[idx] = 1

          resized = interpolate(label_mask, (128,128))
          resized_np_mask[label-1] = resized.copy()
          #guardar array
      

        np.save(output_file_name + "/" + str(contador) , resized_np_mask)
        contador = contador + 1
      contador=0
      contador_path = contador_path +1
  contador_path=0




segment_image("/content/drive/MyDrive/Segmentacion/imagen","/content/drive/MyDrive/Segmentacion/imagen_seccionado_64x64")
segment_label("/content/drive/MyDrive/Segmentacion/label","/content/drive/MyDrive/Segmentacion/label_seccionado_64x64")



<h1> Hacer una limpieza de las imagenes que aportan informacion util para entrenamiento

In [ ]:
dataset = ImagesCustomDataset("/content/drive/MyDrive/Segmentacion/Imagen_Seccionado_128x128","/content/drive/MyDrive/Segmentacion/Label_Seccionado_128x128")

contador = 0
for i in range(len(dataset)):
  file_image_name = '/content/drive/MyDrive/Segmentacion/Imagen_Seccionado_128x128/volume-30/' + str(contador) + ".jpg"
  file_label_name = "/content/drive/MyDrive/Segmentacion/Label_Seccionado_128x128/volume-30"+ "/" + str(contador)
  x,y = dataset[i]
  if np.sum(y[1]) > 0:
    np.save(file_label_name , y)
    x.save(file_image_name)
    contador = contador + 1
  